In [1]:
import cv2
import time
import numpy as np
from pygame import mixer 
import dlib # python api for face recognition model
from math import hypot # to get the length of hor and vert lines we have used hypot function from math lib

mixer.init()
sound = mixer.Sound('Buzz1.wav') #sound for the alarm

cap= cv2.VideoCapture(0) # capture video with value 0 because we are using laptop's webcam
font = cv2.FONT_HERSHEY_COMPLEX_SMALL #set the font type of the text displayed

detector = dlib.get_frontal_face_detector() # we have created an object "detector" which will return the default face detector.
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") 
# we have downloaded shape predictor 68 face shape predictor which runs on the input image
#and returns a single full_object_detection.

blink = 0
TOTAL = 0
Thresh = 5
Consec_frames= 10
Counter = 0
def midpoint(p1 ,p2):  #basic function to find the midpoint
          return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)
    
def get_blinking_ratio(eye_points, facial_landmarks): # funtion to detect the blinking ratio.
    
            left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y) # 0 is the index value of 42
            right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y) # 3 is the index value of 45
            center_top = midpoint(facial_landmarks.part(eye_points[1]), facial_landmarks.part(eye_points[2])) # 1, 2 are the index value of43,44
            center_bottom = midpoint(facial_landmarks.part(eye_points[5]), facial_landmarks.part(eye_points[4])) # 5,4 are the index value of 47,46
            
            hor_line = cv2.line(frame, left_point, right_point, (0, 255, 0), 2)
            ver_line = cv2.line(frame, center_top, center_bottom, (0, 255, 0), 2)
                
            # this will draw horizontal and vertical line on the eye    
            hor_line_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
            ver_line_length = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))          
            
            ratio = hor_line_length / ver_line_length # finding ratio
            return ratio
        
while True: 
    ret, frame = cap.read()
    gray= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # here we are converting frame from BGR to GRAY
    
    faces= detector(gray) # the object faces will detect the frontal face in GRAY scale. 
    
    for face in faces:
        
        landmarks = predictor(gray, face) # convert the resultant landmarks to a NumPy array
        # detect blinking
        left_eye_ratio = get_blinking_ratio([36,37,38,39,40,41], landmarks) 
      
        right_eye_ratio = get_blinking_ratio([42,43,44,45,46,47], landmarks) 
        
        cv2.putText(frame, "left ratio: {:.2f}".format(left_eye_ratio), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv2.putText(frame, "right ratio: {:.2f}".format(right_eye_ratio), (500, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        if left_eye_ratio > Thresh and right_eye_ratio > Thresh:# left eye and right eye ratio is compared with the thresh i.e,5.
            if blink == 0 : # under nested if condition the timer will start when the eye is open
                start_time = time.time()
                blink = 1 # when the first blink happens
            if time.time() - start_time > 0.3: # if time is exceeding the average eye closing time which is 0.3 seconds
                Counter += 1 # then the frame counter will exceed by one
            if Counter >= Consec_frames:# when the counter >= Consec_frames which is defined  with value 10
                sound.play() #play the alarm
                cv2.putText(frame,'ALERT!!!',(100,100), font, 4,(0,0,255),4,cv2.LINE_AA) # throw ALERT message.
                
        else:
            Counter = 0
            
        cv2.imshow("Frame", frame) # to display the resulting frame
    
    if cv2.waitKey(30) & 0xFF == ord('q'): #wait time is 30 sec and if will press key q the video will be stopped.
        break # and get back to while loop again with new frame.

        
cap.release() # when everything is fine it will relaese the video
cv2.destroyAllWindows()      



pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
